In [ ]:
!pip install elasticsearch

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import numpy as np
import re
import os
import pandas as pd
import csv
import json
from typing import List
from elasticsearch import Elasticsearch
from pprint import pprint


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
 sentence = pd.read_csv (r'/content/drive/My Drive/Corpus_project/basic_cleaning_tweets.csv')
 sentence = sentence.rename(columns={'Unnamed: 0': 'number'})
 sentence

## Перегон в JSON

In [ ]:
csvfile = open('/content/drive/My Drive/Corpus_project/basic_cleaning_tweets.csv', 'r')
jsonfile = open('/content/drive/My Drive/Corpus_project/basic_cleaning_tweets.json', 'w')

fieldnames = ("	Unnamed: 0	","id","date","tweet")
reader = csv.DictReader( csvfile, fieldnames)
for row in reader:
    json.dump(row, jsonfile)
    jsonfile.write('\n')

## Настройка ES

In [ ]:
import requests
res = requests.get('http://localhost:9200')
print(res.content)

In [ ]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [ ]:
es.index(index='test-index', doc_type='test', id=1, body={'test': 'test'})

In [ ]:

#delete test data and try with something more interesting
es.delete(index='test-index', doc_type='test', id=1)

## Generator from df

```
# Выбран кодовый формат
```



In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
es_client = Elasticsearch(http_compress=True)


In [ ]:
sentence_iter = sentence.iterrows()
index, document = next(sentence_iter)

In [ ]:
use_these_keys = ['number', 'id', 'date', 'tweet']
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

In [ ]:
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": 'toxic_tweets',
                "_type": "_doc",
                "_id" : f"{document['number']}",
                "_content": f{document['tweet']}
                "_date": f{document['date']}
                "_source": filterKeys(document),
            }
    raise StopIteration
helpers.bulk(es_client, doc_generator(sentence))

## Search

In [ ]:
query = raw_input("Search for tweets: \n")

if __name__ == '__main__':

    es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
    res = es.search(index="searchindex", doc_type="users", body={"query": {"match": {"job": query}}}, size=20)
    print("%d documents found:" % res['hits']['total'])
    for doc in res['hits']['hits']:
        print("%s %s %s" % (doc['_id'], doc['_source']['tweet'], doc['_source']['date']))